In [1]:
#生成随机三维tensor
import os

# 设置可见的 GPU，例如只使用 GPU 0 和 GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
import pandas as pd
df = pd.read_parquet('df.parquet')#[-500:]

In [4]:
horizon_len = 14
patch_len = 32
patch_num = 5
batch_size = 32
hidden_size = 128
channels = ['Close', 'Volume'] #['Close', 'High','Low',	'Open',	'Volume']
device = 'cuda'

In [5]:
from deepdow.utils import raw_to_Xy


n_timesteps = len(df)  # 20
n_channels = len(channels)  # 2
n_assets = len(df.columns.levels[0])  # 2

lookback, gap, horizon = patch_len*patch_num, 2, horizon_len

X, timestamps, y, asset_names, indicators = raw_to_Xy(df,
                                                      lookback=lookback,
                                                      gap=gap,
                                                      freq="B",
                                                      horizon=horizon,
                                                      included_indicators=channels)

n_samples =  n_timesteps - lookback - horizon - gap + 1  # 10

# assert X.shape == (n_samples, n_channels, lookback, n_assets)
# assert timestamps[0] == raw_df.index[lookback]
# assert asset_names == tickers
# assert indicators == ['Close', 'High', 'Low', 'Open', 'Volume']

In [6]:
split_ix = int(n_samples * 0.95)
indices_train = list(range(split_ix))
#indices_test = list(range(split_ix + lookback + horizon, n_samples))
indices_test = list(range(split_ix + horizon, n_samples))

print('Train range: {}:{}\nTest range: {}:{}'.format(indices_train[0], indices_train[-1],
                                                     indices_test[0], indices_test[-1]))

Train range: 0:4374
Test range: 4389:4605


In [7]:
from deepdow.data import InRAMDataset, RigidDataLoader, prepare_standard_scaler, Scale
means, stds = prepare_standard_scaler(X, indices=indices_train)
print('mean: {}, std: {}'.format(means, stds))

mean: [ 0.00038663 -0.00016372], std: [0.01768258 0.34204269]


In [8]:
dataset = InRAMDataset(X, y, timestamps = timestamps, asset_names = asset_names, transform=Scale(means, stds))

In [9]:
dataloader_train = RigidDataLoader(dataset,
                                   indices=indices_train,
                                   batch_size=batch_size)

dataloader_test = RigidDataLoader(dataset,
                                  indices=indices_test,
                                  batch_size=batch_size)

In [10]:
import timesfm
# For Torch
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
          use_positional_embedding=False,
          context_len=768,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.


/home/userroot/miniconda3/envs/ts/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded PyTorch TimesFM, likely because python version is 3.11.11 (main, Dec 11 2024, 16:28:39) [GCC 11.2.0].


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 23537.06it/s]


In [11]:
import torch

In [12]:
for batch_ix, (
                    X_batch,
                    y_batch,
                    timestamps,
                    asset_names,
                ) in enumerate(dataloader_train):
    print("call")
    X = X_batch.to('cuda').to(torch.float) # B, channels, lookback, n_assets
    X = X.permute(0, 3, 1, 2)
    X = X.reshape(-1, X.shape[3])
    input_padding = torch.zeros_like(X).to('cuda')
    freq = torch.tensor([0], dtype=torch.long).to('cuda')
    tfm._model(X, input_padding, freq)

call


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 6.81 MiB is free. Including non-PyTorch memory, this process has 23.68 GiB memory in use. Of the allocated memory 22.81 GiB is allocated by PyTorch, and 586.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#del X
torch.cuda.empty_cache()